In [32]:
import polars as pl

DATA_PATH = "../../data/"

train = pl.read_csv(DATA_PATH + "train.csv")
predicted_prompt = pl.read_csv(DATA_PATH + "predicted_prompt.csv")

In [33]:
predicted_prompt

essay_id,prompt_name,predicted
str,str,bool
"""000fe60""","""The Face on Mars""",true
"""001ab80""","""Driverless cars""",true
"""001bdc0""","""Exploring Venus""",true
"""0033037""","""Facial action coding system""",true
"""0065bd6""","""Driverless cars""",true
…,…,…
"""ffc11a8""","""""A Cowboy Who Rode the Waves""""",false
"""ffc9095""","""Exploring Venus""",false
"""ffddf1f""","""Facial action coding system""",false


In [34]:
train

essay_id,full_text,score
str,str,i64
"""000d118""","""Many people have car where the…",3
"""000fe60""","""I am a scientist at NASA that …",3
"""001ab80""","""People always wish they had th…",4
"""001bdc0""","""We all heard about Venus, the …",4
"""002ba53""","""Dear, State Senator This is a…",3
…,…,…
"""ffd378d""","""the story "" The Challenge of E…",2
"""ffddf1f""","""Technology has changed a lot o…",4
"""fff016d""","""If you don't like sitting arou…",2


In [35]:
train = train.join(  # prompt_nameを付与する
    predicted_prompt.select(pl.col(["essay_id", "prompt_name"])),
    how="left",
    on="essay_id",
)

In [36]:
from datasets import load_dataset

# 追加データ
persuade_dataset = (
    load_dataset(
        "csv",
        data_files={"train": f"{DATA_PATH}/persuade_w_is_tr_con_as_num.csv"},
        split="train",
    )
    .filter(lambda x: not x["is_train_contains"])
    .select_columns(
        ["essay_id_comp", "full_text", "holistic_essay_score", "prompt_name"]
    )
    .rename_columns({"essay_id_comp": "essay_id", "holistic_essay_score": "score"})
)

persuade_df = pl.DataFrame(persuade_dataset.to_pandas())

In [37]:
all_data = pl.concat([train, persuade_df])

In [38]:
all_data.head()

essay_id,full_text,score,prompt_name
str,str,i64,str
"""000d118""","""Many people have car where the…",3,"""Car-free cities"""
"""000fe60""","""I am a scientist at NASA that …",3,"""The Face on Mars"""
"""001ab80""","""People always wish they had th…",4,"""Driverless cars"""
"""001bdc0""","""We all heard about Venus, the …",4,"""Exploring Venus"""
"""002ba53""","""Dear, State Senator This is a…",3,"""Does the electoral college wor…"


In [43]:
essay_id_prompt_dict = dict(zip(all_data["essay_id"], all_data["prompt_name"]))

import json

with open("../../data/essay_id_prompt_dict.json", "w") as f:
    json.dump(essay_id_prompt_dict, f)

In [40]:
len(essay_id_prompt_dict)

30431